In [4]:
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

StatementMeta(, cca63e9f-b7cd-4967-a0dd-c172dbb26cf6, 6, Finished, Available, Finished)

### Get the workspace id

In [5]:
workspaceId = spark.conf.get('trident.workspace.id')
print(f"{workspaceId=}")

StatementMeta(, cca63e9f-b7cd-4967-a0dd-c172dbb26cf6, 7, Finished, Available, Finished)

workspaceId='347a7e22-a2b5-4a16-85f1-e763f6b7f66b'


In [6]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', headers=header)
workspaceName = response.json().get('displayName')
print(f'{workspaceName=}')

StatementMeta(, cca63e9f-b7cd-4967-a0dd-c172dbb26cf6, 8, Finished, Available, Finished)

workspaceName='WS_TESTING_DEPLOY_3'


### Create lakehouses

In [4]:
from notebookutils import mssparkutils

for lakehouseName in ['LH_SampleData', 'LH_QueryResults']:
    try:
        lakehouseItem = mssparkutils.lakehouse.create(lakehouseName, "", workspaceId)
    except Exception as e:
        # print(e)
        pass

LH_SampleData = mssparkutils.lakehouse.get("LH_SampleData", workspaceId)
LH_QueryResults = mssparkutils.lakehouse.get("LH_QueryResults", workspaceId)
print(f'{LH_SampleData=}')
print(f'{LH_QueryResults=}')

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 6, Finished, Available, Finished)

LH_SampleData={'id': 'cff7fdc6-e7de-4c66-871c-2b58f0984f45', 'type': 'Lakehouse', 'displayName': 'LH_SampleData', 'description': '', 'workspaceId': '347a7e22-a2b5-4a16-85f1-e763f6b7f66b', 'properties': {'abfsPath': 'abfss://347a7e22-a2b5-4a16-85f1-e763f6b7f66b@onelake.dfs.fabric.microsoft.com/cff7fdc6-e7de-4c66-871c-2b58f0984f45'}}
LH_QueryResults={'id': 'f3a825ac-3233-4023-9c1f-f6878b7e4f67', 'type': 'Lakehouse', 'displayName': 'LH_QueryResults', 'description': '', 'workspaceId': '347a7e22-a2b5-4a16-85f1-e763f6b7f66b', 'properties': {'abfsPath': 'abfss://347a7e22-a2b5-4a16-85f1-e763f6b7f66b@onelake.dfs.fabric.microsoft.com/f3a825ac-3233-4023-9c1f-f6878b7e4f67'}}


### Get notebook definition from github

In [5]:
import requests, json, base64
url = "https://raw.githubusercontent.com/bretamyers/FabricTools/dev/FabricDWQueryCostAnalyzer/NB_DW_Load_Cost_Analyzer.ipynb"
response = requests.get(url)

b = base64.b64encode(bytes(json.dumps(response.json()), 'utf-8')) # convert to bytes
base64_str = b.decode('utf-8') # convert bytes to string
notebookDefDict = json.loads(base64.b64decode(base64_str).decode('utf-8')) # convert base64 bytes to string and then to dictionary
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse'] = LH_QueryResults.get('id')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_name'] = LH_QueryResults.get('displayName')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_workspace_id'] = LH_QueryResults.get('workspaceId')

for cell in notebookDefDict['cells']:
    if cell['id'] == "20be0399-9430-4b52-a3ab-1c911fc0d69a":
        cell['source'] = [
                f"FabricDWWorkspaceName = '{workspaceName}'\n",
                "FabricDWName = 'WH_SampleData'\n",
                "ConcurrencyNum = 1 # This should be equal or greater than the length of the dataframe with the queryies defined below\n",
                f"CapacityMetricsWorkspace = '{CapacityMetricsWorkspace}'\n",
                f"CapacityMetricsDataset = '{CapacityMetricsDataset}'"
            ]
notebookDefBase64 = base64.b64encode(json.dumps(notebookDefDict).encode('utf-8')).decode('utf-8') # convert dictionary back to base64 string
# print(json.dumps(notebookDefDict, indent=2))

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 7, Finished, Available, Finished)

### Create or update notebook from the definition

In [6]:
import requests, json, time, base64

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

notebookFound = False
if response.status_code == 200:
    # Check if notebook already exists
    for notebook in response.json().get('value'):
        if notebook.get('displayName') == 'NB_DW_Load_Cost_Analyzer':
            notebookFound = True
            body = {"definition": {
                "format": "ipynb",
                "parts": [
                    {
                        "path": "notebook-content.py"
                        ,"payload": notebookDefBase64
                        ,"payloadType": "InlineBase64"
                    }
                ]
                }}
            response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks/{notebook.get("id")}/updateDefinition',  headers=header, data=json.dumps(body))

            if response.status_code == 202:
                responseStatus = response
                for retry in range(5):
                    time.sleep(int(responseStatus.headers.get('Retry-After')))
                    responseStatus = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                    if responseStatus.json().get('status') == 'Succeeded':
                        break
                
if not notebookFound:
    print('Not found')
    body = {
        "displayName": "NB_DW_Load_Cost_Analyzer",
        "description": "A notebook to run data warehouse queries and capture the duration and cost of those queries.",
        "definition": {
            "format": "ipynb",
            "parts": [
                {
                    "path": "notebook-content.py"
                    ,"payload": notebookDefBase64
                    ,"payloadType": "InlineBase64"
                }
            ]
        }
    }

    response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header, data=json.dumps(body))

    if response.status_code == 202:
        time.sleep(int(response.headers.get('Retry-After')))
        for retry in range(5):
            responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
            print(responseStatus.text)
            print(responseStatus.headers)
            if responseStatus.headers.get('Location') is not None:
                break
            else:
                time.sleep(int(responseStatus.headers.get('Retry-After')))
    else:
        print(f"Error - {response.json()}")

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 8, Finished, Available, Finished)

### Get the notebook id

In [7]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
          
# Need to implement for pagination
response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

for notebook in response.json().get('value'):
    if notebook.get('displayName') == 'NB_DW_Load_Cost_Analyzer':
        notebookId = notebook.get('id')
        print(f'{notebookId=}')


StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 9, Finished, Available, Finished)

notebookId='6d40553f-1bf6-4750-9964-97b862ea45cd'


### Create Warehouse

In [8]:
import requests, json, time

body = {"displayName": "WH_SampleData"}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print(f'Creating Warehouse {body.get("displayName")}')
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.json().get('status') != 'Succeeded':
            time.sleep(int(responseStatus.headers.get('Retry-After')))
        else:
            if responseStatus.headers.get('Location') is None: # no result was return but operation completed
                break
            else:
                responseResult = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                print(f"Succeeded {responseResult.json()}")
                break
else:
    print(response.text)

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 10, Finished, Available, Finished)

{"requestId":"63c5a04c-f6aa-43c7-ad53-d23a6f758213","errorCode":"UnknownError","message":"An unexpected error occurred while processing the request"}


In [9]:
import requests, json, time

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header)

for warehouse in response.json().get('value'):
    if warehouse.get('displayName') == 'WH_SampleData':
        WH_SampleData = warehouse
        break
print(WH_SampleData)

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 11, Finished, Available, Finished)

{'id': '2d562264-57f1-4bf2-af85-6414fc22f3df', 'type': 'Warehouse', 'displayName': 'WH_SampleData', 'description': '', 'workspaceId': '347a7e22-a2b5-4a16-85f1-e763f6b7f66b', 'properties': {'connectionInfo': 'n52dzjnhphme5jslxpytuo62ni-ej7hunfvuilevbpr45r7nn7wnm.datawarehouse.fabric.microsoft.com', 'connectionString': 'n52dzjnhphme5jslxpytuo62ni-ej7hunfvuilevbpr45r7nn7wnm.datawarehouse.fabric.microsoft.com', 'createdDate': '2024-09-25T17:29:28.523', 'lastUpdatedTime': '2024-09-25T17:29:42.35'}}


### Load data to Lakehouse

In [10]:
import azure.storage.blob
import pandas as pd
from notebookutils import mssparkutils

spark.conf.set('spark.sql.parquet.int96RebaseModeInWrite', 'LEGACY')
spark.conf.set('spark.sql.parquet.datetimeRebaseModeInWrite', 'LEGACY')

account_url = "https://fabrictutorialdata.blob.core.windows.net"
blob_service_client = azure.storage.blob.BlobServiceClient(account_url)
container_client = blob_service_client.get_container_client('sampledata')

tableList = list()
for blob in container_client.walk_blobs(name_starts_with='WideWorldImportersDW/parquet/tables/', delimiter='/'):
    tableName = blob.name.split('/')[-1].split('.parquet')[0]
    tableList.append(tableName)

for table in tableList:
    print(f'Loading Table: {table}')
    if mssparkutils.fs.exists(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}"):
        mssparkutils.fs.rm(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}", recurse=True)

    for blob in container_client.walk_blobs(name_starts_with=f"WideWorldImportersDW/parquet/tables/", delimiter='/'):
        if f'{table}.parquet' == blob.name.split('/')[-1]:
            dfPandas = pd.read_parquet(f'{account_url}/sampledata/{blob.name}', engine='pyarrow', storage_options={'anon': True})

            if not dfPandas.empty:
                dfSpark = spark.createDataFrame(dfPandas)
                dfSpark.write.format('delta').mode('append').option('mergeSchema', "true").save(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}")
            else:
                print('Bad file', blob.name)

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 12, Finished, Available, Finished)

Loading Table: DimCity
Loading Table: DimCustomer
Loading Table: DimDate
Loading Table: DimEmployee
Loading Table: DimPaymentMethod
Loading Table: DimStockItem
Loading Table: DimSupplier
Loading Table: DimTransactionType
Loading Table: FactMovement
Loading Table: FactOrder
Loading Table: FactPurchase
Loading Table: FactSale
Loading Table: FactStockHolding
Loading Table: FactTransaction


### Load data to Warehouse

In [11]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        for retry in range(5):
            cursor.execute(f"""SELECT TABLE_NAME FROM LH_SampleData.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo'""")
            resultSet = get_result_set(cursor)
            if len(resultSet) == 14:
                break
            else:
                time.sleep(30) # wait 30 seconds for the schema sync to occur
        while cursor.nextset():
            queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
            resultSet.append(get_result_set(cursor))
        cursor.commit()

    for table in resultSet:
        with conn.cursor() as cursor:
            query = f"""IF OBJECT_ID('dbo.{table.get("TABLE_NAME")}', 'U') IS NOT NULL DROP TABLE dbo.{table.get("TABLE_NAME")}; CREATE TABLE dbo.{table.get("TABLE_NAME")} AS SELECT * FROM LH_SampleData.dbo.{table.get("TABLE_NAME")}"""
            print(query)
            cursor.execute(query)
            cursor.commit()


StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 13, Finished, Available, Finished)

IF OBJECT_ID('dbo.FactOrder', 'U') IS NOT NULL DROP TABLE dbo.FactOrder; CREATE TABLE dbo.FactOrder AS SELECT * FROM LH_SampleData.dbo.FactOrder
IF OBJECT_ID('dbo.FactTransaction', 'U') IS NOT NULL DROP TABLE dbo.FactTransaction; CREATE TABLE dbo.FactTransaction AS SELECT * FROM LH_SampleData.dbo.FactTransaction
IF OBJECT_ID('dbo.DimCustomer', 'U') IS NOT NULL DROP TABLE dbo.DimCustomer; CREATE TABLE dbo.DimCustomer AS SELECT * FROM LH_SampleData.dbo.DimCustomer
IF OBJECT_ID('dbo.DimSupplier', 'U') IS NOT NULL DROP TABLE dbo.DimSupplier; CREATE TABLE dbo.DimSupplier AS SELECT * FROM LH_SampleData.dbo.DimSupplier
IF OBJECT_ID('dbo.FactMovement', 'U') IS NOT NULL DROP TABLE dbo.FactMovement; CREATE TABLE dbo.FactMovement AS SELECT * FROM LH_SampleData.dbo.FactMovement
IF OBJECT_ID('dbo.DimStockItem', 'U') IS NOT NULL DROP TABLE dbo.DimStockItem; CREATE TABLE dbo.DimStockItem AS SELECT * FROM LH_SampleData.dbo.DimStockItem
IF OBJECT_ID('dbo.DimPaymentMethod', 'U') IS NOT NULL DROP TABLE d

In [12]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        cursor.execute('''IF OBJECT_ID('dbo.sp_Query', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Query
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Query AS
	SELECT COUNT(*) AS cnt_FactOrder FROM FactOrder

    SELECT COUNT(*) AS cnt_FactMovement FROM FactMovement

    SELECT COUNT(*) AS cnt_FactPurchase FROM FactPurchase

;''')
        cursor.execute('''IF OBJECT_ID('dbo.sp_Ingest', 'P') IS NOT NULL
	DROP PROCEDURE dbo.sp_Ingest
;''')
        cursor.execute('''CREATE PROCEDURE dbo.sp_Ingest AS
        IF OBJECT_ID('dbo.DimCity', 'U') IS NOT NULL DROP TABLE dbo.DimCity; CREATE TABLE dbo.DimCity AS SELECT * FROM LH_SampleData.dbo.DimCity;

        SELECT COUNT(*) FROM DimCity
;''')
        cursor.commit()


StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 14, Finished, Available, Finished)

### Refresh Fabric Capacity Metrics App so that the latest workspace artifacts are in the model

In [2]:
import requests, json, time

def model_refresh():
    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

    response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

    capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
    print(f'{capacityWorkspaceId = }')

    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

    datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
    print(f'{datasetId = }')

    response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

    refreshId = response.headers.get('RequestId')
    print(f'{refreshId = } | {response.status_code = }')

    if response.status_code == 202:
        for attempt in range(12): 
            # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
            response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
            if response.status_code == 200:
                if response.json().get('value')[0].get('status') != 'Unknown':
                    print(f'Refresh Complete')
                    break
                else:
                    print(f'Refreshing tables ...')
                    time.sleep(20)
            else:
                time.sleep(10)
    else:
        print(f'Refreshed failed - {response.text}')

StatementMeta(, cca63e9f-b7cd-4967-a0dd-c172dbb26cf6, 4, Finished, Available, Finished)

In [9]:

import requests, math

def check_if_workspace_exists():
  header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

  response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

  capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
  print(f'{capacityWorkspaceId = }')

  response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

  datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
  print(f'{datasetId = }')

  body = {
    "queries": [
      {
        "query": f"""
            DEFINE
              VAR __DS0Core = 
                DISTINCT('Items'[WorkspaceName])
                
            EVALUATE
              __DS0Core
      """
      }
    ]
  }

  response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header, json=body )

  return response

workspaceFound = False
response = check_if_workspace_exists()
for results in response.json().get('results'):
  for table in results.get('tables'):
    for row in table.get('rows'):
      if row.get('Items[WorkspaceName]') == workspaceName:
        print(f'Workspace "{workspaceName}" was found in the capacity metrics app model')
        workspaceFound = True

if not workspaceFound:
  for retry in range(10):
      model_refresh()
      response = check_if_workspace_exists()
      for results in response.json().get('results'):
          for table in results.get('tables'):
              for row in table.get('rows'):
                  if row.get('Items[WorkspaceName]') == workspaceName:
                      workspaceFound = True
      if workspaceFound:
          print(f'Workspace "{workspaceName}" was found in the capacity metrics app model after doing a model refresh.')
          break
      else:
          print(f'Workspace "{workspaceName}" was not found. Waiting 2 minutes and will refresh the model and try again...')
          time.sleep(120)

StatementMeta(, cca63e9f-b7cd-4967-a0dd-c172dbb26cf6, 11, Finished, Available, Finished)

capacityWorkspaceId = '0315286f-829f-410f-a0aa-1e45af27297c'
datasetId = 'a3443502-d96b-4b3f-99e5-76920052edad'
Workspace "WS_TESTING_DEPLOY_3" was found in the capacity metrics app model


### Run the notebook

In [15]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "executionData": {
        "parameters": {
            "FabricDWWorkspaceName": {"value": workspaceName, "type": "string"}
            ,"FabricDWName": {"value": "WH_SampleData", "type": "string"}
            ,"ConcurrencyNum": {"value": "1", "type": "int"}
            ,"CapacityMetricsWorkspace": {"value": CapacityMetricsWorkspace, "type": "string"}
            ,"CapacityMetricsDataset": {"value": CapacityMetricsDataset, "type": "string"}
        }
    }
}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/jobs/instances?jobType=RunNotebook', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print('Started Execution of Notebook')
    time.sleep(10) # Sleep for 10 seconds to wait for job to get started. TODO Change in the future to a retry
    print(f"{response.headers=}")
    # jobId = response.headers.get('Location')
    while True:
        # print(f"{response.headers.get('Location')=}")
        responseStatus = requests.request(method="get", url=response.headers.get('Location'), headers=header)
        print(f"Status: {responseStatus.json().get('status')} | {responseStatus.json().get('failureReason') if responseStatus.json().get('failureReason') != None else ''}")
        if responseStatus.json().get('status') in ['Succeeded', 'Failed', 'Completed']:
            print(responseStatus.json().get('status'))
            break
        else:
            time.sleep(120) # Wait 2 minutes to check the status of the run
else:
    print('Error trying to execute')

StatementMeta(, a0d59aad-e52d-4cc0-9b0a-6f0672fb225f, 17, Finished, Cancelled, Cancelled)

Started Execution of Notebook
response.headers={'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '0', 'Content-Type': 'application/octet-stream', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/workspaces/347a7e22-a2b5-4a16-85f1-e763f6b7f66b/items/6d40553f-1bf6-4750-9964-97b862ea45cd/jobs/instances/85718c8b-a58f-49bc-a72b-f51187b8f3e6', 'Retry-After': '60', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': 'f30487db-6c3e-433b-baa9-951f35b8f1b0', 'Access-Control-Expose-Headers': 'RequestId,Location,Retry-After', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Wed, 25 Sep 2024 18:14:31 GMT'}
Status: InProgress | 
